<a href="https://colab.research.google.com/github/gitleon8301/MY-AI-Gizmo-working/blob/main/Colab-TextGen-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# oobabooga/text-generation-webui

After running both cells, a public gradio URL will appear at the bottom in around 10 minutes. You can optionally generate an API link.

* Project page: https://github.com/oobabooga/text-generation-webui
* Gradio server status: https://status.gradio.app/

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════
# MY-AI-Gizmo Complete Setup Script for Google Colab
# Repository: https://github.com/gitleon8301/MY-AI-Gizmo-working
# ═══════════════════════════════════════════════════════════════════════════

import os
import sys
import subprocess
from pathlib import Path
import shutil

print("🚀 MY-AI-GIZMO INSTALLER FOR GOOGLE COLAB")
print("="*70)

# --- STEP 1: Clean and Clone ---
print("\n📥 Step 1: Cloning repository...")
os.chdir('/content')

if Path("MY-AI-Gizmo-working").exists():
    print("   Removing existing directory...")
    shutil.rmtree("MY-AI-Gizmo-working")

subprocess.run([
    "git", "clone",
    "https://github.com/gitleon8301/MY-AI-Gizmo-working.git"
], check=True)

os.chdir("MY-AI-Gizmo-working")
print("   ✅ Repository cloned")

# --- STEP 2: Environment Setup ---
print("\n🔧 Step 2: Setting up environment...")
os.environ['MPLBACKEND'] = 'Agg'
os.environ['BITSANDBYTES_NOWELCOME'] = '1'
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
print("   ✅ Environment configured")

# --- STEP 3: Explore Repository Structure ---
print("\n🔍 Step 3: Analyzing repository structure...")

# Check what files exist
key_files = {
    "requirements.txt": Path("requirements.txt"),
    "requirements_cpu_only.txt": Path("requirements_cpu_only.txt"),
    "requirements_cpu_only_windows.txt": Path("requirements_cpu_only_windows.txt"),
    "server.py": Path("server.py"),
    "download-model.py": Path("download-model.py"),
    "start_linux.sh": Path("start_linux.sh"),
    "start_windows.bat": Path("start_windows.bat"),
    "installer_files/requirements.txt": Path("installer_files/requirements.txt"),
}

print("\n   Files found:")
for name, path in key_files.items():
    if path.exists():
        print(f"   ✅ {name}")
    else:
        print(f"   ❌ {name}")

# --- STEP 4: Install Dependencies ---
print("\n📦 Step 4: Installing dependencies...")

# Upgrade pip first
subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "pip"],
               check=False, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Try to find and use the correct requirements file
req_priority = [
    "requirements_cpu_only.txt",
    "requirements.txt",
    "installer_files/requirements.txt"
]

requirements_installed = False
for req_file in req_priority:
    if Path(req_file).exists():
        print(f"   Installing from {req_file}...")
        try:
            subprocess.run([
                sys.executable, "-m", "pip", "install",
                "-r", req_file, "--quiet"
            ], check=True, timeout=300)
            print(f"   ✅ Installed from {req_file}")
            requirements_installed = True
            break
        except (subprocess.CalledProcessError, subprocess.TimeoutExpired) as e:
            print(f"   ⚠️  Failed with {req_file}, trying next...")
            continue

# Fallback: Install essential packages manually
if not requirements_installed:
    print("   📦 Installing core packages manually...")
    core_packages = [
        "torch",
        "torchvision",
        "torchaudio",
        "transformers>=4.30.0",
        "gradio>=3.50.0",
        "accelerate",
        "peft",
        "markdown",
        "numpy",
        "Pillow>=9.5.0",
        "pyyaml",
        "requests",
        "safetensors>=0.3.1",
        "sentencepiece",
        "tqdm",
        "protobuf",
        "llama-cpp-python"
    ]

    for package in core_packages:
        try:
            subprocess.run([
                sys.executable, "-m", "pip", "install",
                package, "--quiet"
            ], check=True, timeout=60)
            print(f"   ✅ {package}")
        except:
            print(f"   ⚠️  {package} (skipped)")

# --- STEP 5: Fix Gradio Theme Issue ---
print("\n🔧 Step 5: Patching UI compatibility...")

ui_file = Path("modules/ui.py")
if ui_file.exists():
    try:
        with open(ui_file, 'r', encoding='utf-8') as f:
            content = f.read()

        # Check if the problematic code exists
        if 'button_shadow_hover' in content or 'theme.set(' in content:
            # Create a backup
            shutil.copy(ui_file, ui_file.with_suffix('.py.bak'))

            # Replace the theme configuration
            content = content.replace(
                'if not shared.args.old_colors:',
                'if False:  # Theme disabled for Colab - was: if not shared.args.old_colors:'
            )

            with open(ui_file, 'w', encoding='utf-8') as f:
                f.write(content)

            print("   ✅ UI theme patched")
        else:
            print("   ℹ️  No theme issues detected")
    except Exception as e:
        print(f"   ⚠️  Could not patch UI: {e}")
else:
    print("   ℹ️  No UI file found to patch")

# --- STEP 6: Setup Models Directory ---
print("\n📁 Step 6: Creating models directory...")

models_dir = Path("models")
models_dir.mkdir(exist_ok=True)
print(f"   ✅ Models directory: {models_dir.absolute()}")

# --- STEP 7: Download Model ---
print("\n⬇️  Step 7: Downloading model...")
print("   This may take 3-5 minutes...")

model_name = "llama-2-7b.Q4_K_M.gguf"
model_path = models_dir / model_name

if model_path.exists():
    print(f"   ℹ️  Model already exists: {model_path}")
else:
    # Try the download script first
    if Path("download-model.py").exists():
        print("   Trying download-model.py...")
        try:
            subprocess.run([
                sys.executable, "download-model.py",
                "TheBloke/Llama-2-7B-GGUF"
            ], check=True, timeout=600)
            print("   ✅ Model downloaded via script")
        except:
            print("   ⚠️  Script failed, trying direct download...")

    # Fallback: Direct download
    if not model_path.exists():
        print("   Direct download from Hugging Face...")
        model_url = "https://huggingface.co/TheBloke/Llama-2-7B-GGUF/resolve/main/llama-2-7b.Q4_K_M.gguf"

        try:
            subprocess.run([
                "wget", "-q", "--show-progress",
                "-O", str(model_path),
                model_url
            ], check=True, timeout=600)
            print(f"   ✅ Model downloaded: {model_path}")
        except Exception as e:
            print(f"   ❌ Download failed: {e}")
            sys.exit(1)

# Verify model exists
if not model_path.exists():
    # Try to find any GGUF file
    gguf_files = list(Path(".").rglob("*.gguf"))
    if gguf_files:
        model_path = gguf_files[0]
        print(f"   ✅ Found model: {model_path}")
    else:
        print("   ❌ No model file found!")
        sys.exit(1)

# --- STEP 8: Launch Configuration ---
print("\n⚙️  Step 8: Preparing launch...")

# Determine the correct launch script
if Path("server.py").exists():
    launch_cmd = [sys.executable, "server.py"]
    print("   Using: server.py")
elif Path("start_linux.sh").exists():
    subprocess.run(["chmod", "+x", "start_linux.sh"], check=False)
    launch_cmd = ["bash", "start_linux.sh"]
    print("   Using: start_linux.sh")
else:
    print("   ❌ No launch script found!")
    sys.exit(1)

# Add arguments
launch_args = [
    "--cpu",  # Force CPU mode
    "--listen",  # Listen on all interfaces
    "--share",  # Create public Gradio link
    "--model", str(model_path),  # Model file
    "--chat",  # Enable chat mode
    "--verbose"  # Verbose output
]

full_command = launch_cmd + launch_args

# --- STEP 9: Launch Server ---
print("\n" + "="*70)
print("🚀 LAUNCHING MY-AI-GIZMO SERVER")
print("="*70)
print(f"Model: {model_path}")
print(f"Mode: CPU-only")
print(f"Interface: Gradio Web UI")
print(f"\nCommand: {' '.join(full_command)}")
print("="*70)
print("\n⏳ Starting server (this may take 1-2 minutes)...")
print("🔗 Look for the Gradio link below (https://xxxxx.gradio.live)\n")

try:
    subprocess.run(full_command, check=True)
except KeyboardInterrupt:
    print("\n\n⏹️  Server stopped by user")
except Exception as e:
    print(f"\n\n❌ Server error: {e}")
    print("\n🔍 Debugging information:")
    print(f"Working directory: {os.getcwd()}")
    print(f"Python: {sys.executable}")
    print(f"Model exists: {model_path.exists()}")

    # Show recent files
    print("\n📂 Recent files in directory:")
    subprocess.run(["ls", "-lah"], check=False)